In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#clasificadores
from utils.clasificacion import *

#adaptacion
from utils.adaptacion import msda_theano_grid_search

#otros
import os
import numpy as np
import pandas as pd
import timeit
from sklearn.externals import joblib

tipo = pruebas[1]
dataset_name = datasets[0]
dims = dimensions[dataset_name]

Using Theano backend.
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Pruebas con el dataset Amazon

In [3]:
print tipo
print dataset_name
print dims
print data_path

msda
amazon
1000
data


In [4]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


# Adaptación

## Creación de modelos de adaptación.

Por cada par de dominios se entrenan distintos modelos según los parámetros establecidos.
Al entrenar cada modelo, este es inmediatamente probado, siendo almacenado o descartado según su desempeño.

Cada modelo es guardado en la ruta: models/amazon/msda/me2\_[dominio\_fuente]_[dominio\_objetivo].pkl

### Creación de modelos de adaptación.

In [6]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me2_models_paths.pkl')

parameters = {
    'noises': [0.3, 0.5, 0.8],
    'layers': [1, 3, 5]
}

i = 1

#por cada par se entrena un adaptador y se guarda el que mejor resultados entrega
print "Adaptando entre:"
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "%d) %s - %s" % (i, src, tgt)
            X_src = np.asarray(labeled[src]['X_tr'].todense())
            y_src = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

            X_tgt = np.asarray(labeled[tgt]['X_tr'].todense())
            
            X_train = np.concatenate((X_src, X_tgt))
            
            best_model, score = msda_theano_grid_search(X_train, X_src, y_src, parameters, n_jobs=4)
            print "\tMejor modelo: %.3f" % score
            
            folder_path = os.path.join(models_path, dataset_name, tipo)
            prefix = "me2_" + src + "_" + tgt + ".pkl"
            
            best_model_save_path = os.path.join(folder_path, prefix)
            print "\tGuardando modelo en %s" % best_model_save_path
            joblib.dump(best_model, best_model_save_path)
            i = i+1

print "\nAdaptaciones completadas"

Adaptando entre:
kitchen - dvd
	pr: 0.300 - l: 1
	Entrenando capa  1
	Entrenado en 0.09m

0.884640200179
	Mejor modelo: 0.885
	Guardando modelo en models/amazon/msda/me2_kitchen_dvd.pkl
kitchen - electronics
	pr: 0.300 - l: 1
	Entrenando capa  1
	Entrenado en 0.11m

0.892248826706
	Mejor modelo: 0.892
	Guardando modelo en models/amazon/msda/me2_kitchen_electronics.pkl
kitchen - books
	pr: 0.300 - l: 1
	Entrenando capa  1
	Entrenado en 0.10m

0.89243606043
	Mejor modelo: 0.892
	Guardando modelo en models/amazon/msda/me2_kitchen_books.pkl
dvd - kitchen
	pr: 0.300 - l: 1
	Entrenando capa  1
	Entrenado en 0.10m

0.823327949364
	Mejor modelo: 0.823
	Guardando modelo en models/amazon/msda/me2_dvd_kitchen.pkl
dvd - electronics
	pr: 0.300 - l: 1
	Entrenando capa  1
	Entrenado en 0.12m

0.823617887601
	Mejor modelo: 0.824
	Guardando modelo en models/amazon/msda/me2_dvd_electronics.pkl
dvd - books
	pr: 0.300 - l: 1
	Entrenando capa  1
	Entrenado en 0.12m

0.817419151658
	Mejor modelo: 0.817
	Gua

## Pruebas de clasificación

In [7]:
df = pd.DataFrame(columns=dataframe_columns)

i=1
tareas = len(domains)*(len(domains)-1)

# por cada par posible para adaptar
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de %d" % (i, tareas)
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            model_name = "%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, "indomain", model_name)
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            #############
            #### mSDA ###
            #############
            # se adaptan los dominios usando mSDA
            print "Adaptando dominios..."
            folder_path = os.path.join(models_path, dataset_name, tipo)
            prefix = "me2_" + src + "_" + tgt + ".pkl"
            
            best_model_save_path = os.path.join(folder_path, prefix)
            
            best_msda = joblib.load(best_model_save_path)
            
            
            X_tr = np.asarray(labeled[src]['X_tr'].todense())
            X_tr_a = best_msda.predict(X_tr)
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            X_ts_a = best_msda.predict(X_ts)
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            clf = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
            t_error = 1-clf.score(X_ts_a, y_ts)
            
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['mSDA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            
            i += 1
            
print "\nPruebas completadas."

Tarea 1 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 2 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 3 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 4 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 5 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 6 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 7 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 8 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 9 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 10 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 11 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 12 de 12
Cargando modelo existente.
Adaptando dominios...

Pruebas completadas.


In [8]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
1,mSDA,k->d,kitchen,dvd,26.25,34.268357,8.018357
2,mSDA,k->e,kitchen,electronics,23.50,16.082902,-7.417098
3,mSDA,k->b,kitchen,books,25.50,34.263357,8.763357
4,mSDA,d->k,dvd,kitchen,20.75,19.797995,-0.952005
5,mSDA,d->e,dvd,electronics,23.50,29.580740,6.080740
6,mSDA,d->b,dvd,books,25.50,25.620641,0.120641
7,mSDA,e->k,electronics,kitchen,20.75,10.962774,-9.787226
8,mSDA,e->d,electronics,dvd,26.25,29.765744,3.515744
9,mSDA,e->b,electronics,books,25.50,30.725768,5.225768
10,mSDA,b->k,books,kitchen,20.75,22.858071,2.108071


In [11]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me2_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/amazon/msda/me2_1000.csv
Resultados guardados.
